In [25]:
import xgboost as xgb
import numpy as np
import pandas as pd
from scipy.stats import rankdata

In [26]:

# Load the model
rank_model = xgb.Booster()
rank_model.load_model("xgboost_score_model.json")

In [27]:
# Normalize each feature (column) using Min-Max scaling
def custom_normalization(features):
    # Initialize an empty list to hold the normalized data
    normalized_data = np.zeros_like(features, dtype=float)
    total_distance = features[0]
    num_of_wps = features[1]
    max_score  = num_of_wps*10
    max_num_of_wps = total_distance//10

    for i in range(6):
        feature_val = features[i]
        
        # If it's distance or the battery level, divide by 100
        if i == 0 or i == 5:
            normalized_data[i] = feature_val / 100
            
        elif i == 1:
            normalized_data[i] = feature_val / max_num_of_wps
        else:
                      
            # Normalize the feature to the range [0, 1]
            if max_score > 0:  # Avoid division by zero if all values are the same
                normalized_data[i] = feature_val / max_score
            else:
                normalized_data[i] = 0  # If all values are the same, assign 0 (or handle as needed)
    
    return np.round(normalized_data, 4)

def normalize_five_feature_sets(five_feature_sets):
    normalized_data = np.zeros_like(five_feature_sets, dtype=float)
    
    for i in range(5):
        feature_set = five_feature_sets[i]
        
        normalized_data[i] = custom_normalization(feature_set)
    
    return normalized_data

In [28]:
data = [
    [28, 2, 15,  5, 15, 43],
    [37, 2, 11, 15,  0, 43],
    [31, 2, 10, 10, 10, 43],
    [32, 2, 16, 10,  5, 43],
    [69, 4, 10, 20, 15, 43],
]

nrom_data = normalize_five_feature_sets(data)

# Set column names
column_names = ["Batt", "Dist", "Wpts", "Emot", "Comf", "Traf"]

# Convert to DataFrame
dataframe = pd.DataFrame(nrom_data, columns=column_names)
dataframe

,Batt,Dist,Wpts,Emot,Comf,Traf
0,0.28,1.0000,0.75,0.25,0.750,0.43
1,0.37,0.6667,0.55,0.75,0.000,0.43
2,0.31,0.6667,0.50,0.50,0.500,0.43
3,0.32,0.6667,0.80,0.50,0.250,0.43
4,0.69,0.6667,0.25,0.50,0.375,0.43


In [29]:
# Create DMatrix
dtest = xgb.DMatrix(dataframe)

# Predict scores
y_pred = rank_model.predict(dtest)

# Print the predicted scores
print("Predicted Scores:", y_pred)


Predicted Scores: [-1.4072024  1.072507   0.3639794  1.0762063 -1.9035856]


In [33]:
# Number of items per query group
group_size = 5  # Adjust this to match your data

# Rank predictions for each query group
num_groups = len(y_pred) // group_size

for i in range(num_groups):
    start = i * group_size
    end = (i + 1) * group_size
    
    # Get predictions for the current group
    query_results = y_pred[start:end]
    
    # Rank in descending order
    ranked_positions = rankdata(-query_results, method="ordinal")
    
    print(f"Group {i + 1}:")
    print("Predicted Scores:", query_results)
    print("Predicted Ranks:", ranked_positions)
    print()


Group 1:
Predicted Scores: [-1.4072024  1.072507   0.3639794  1.0762063 -1.9035856]
Predicted Ranks: [4 2 3 1 5]



In [31]:

# Define the penalty or reward calculation for each feature
def calculate_distance_penalty(total_distance, battery_level):
    # Define how the distance and battery level affect the penalty
    # Assuming the battery level reduces penalty when higher
    return -4*(total_distance**2) * (1.1 - battery_level) # Example of distance penalty with battery adjustment

def calculate_waypoint_penalty(number_of_waypoints):
    # Negative log transformation for waypoints to penalize high waypoint count
    return -np.log(number_of_waypoints + 1)**3  # Penalty for each waypoint

def calculate_emotion_benefit(emotion_score):
    # If emotion_score is lower than 0.4, return a negative value
    if emotion_score < 0.4:
        return -(1.1-emotion_score)**2  # Penalty for lower emotion score
    else:
        return (emotion_score*1.5)  # Reward for higher emotion score

def calculate_comfort_benefit(comfort_score):
    # Comfort score is positive, so we give a benefit to higher comfort scores
    if comfort_score < 0.4:
        return -(1.1-comfort_score)  # Penalty for lower emotion score
    else:
        return  comfort_score # Reward for higher comfort score

def calculate_traffic_penalty(traffic_score):
    # Traffic score is penalized, so we penalize higher traffic scores
    return -2*traffic_score**2  # Exponential penalty for traffic, adjust factor as needed

def calculate_path_score(feature_set):
    total_distance, number_of_waypoints, emotion_score, comfort_score, traffic_score, battery_level = feature_set
    
    # Calculate the penalty or reward for each feature
    distance_penalty = calculate_distance_penalty(total_distance, battery_level)
    waypoint_penalty = calculate_waypoint_penalty(number_of_waypoints)
    emotion_benefit = calculate_emotion_benefit(emotion_score)
    comfort_benefit = calculate_comfort_benefit(comfort_score)
    traffic_penalty = calculate_traffic_penalty(traffic_score)
    
    # Combine all features using their respective weights
    score = (distance_penalty +
             waypoint_penalty +
             emotion_benefit +
             comfort_benefit +
             traffic_penalty)
    
    return np.round(score, 8)


In [32]:
# Print path scores
for i in range(len(norm_data)):
    print(f"Path {i+1} Score: {calculate_path_score(norm_data[i])}")

Path 1 Score: -1.39313665
Path 2 Score: 1.07479607
Path 3 Score: 0.35914007
Path 4 Score: 1.16725607
Path 5 Score: -1.91300993
